In [17]:
import os

import numpy as np
import pandas as pd
import json

from sklearn.metrics import cohen_kappa_score
import krippendorff as kd

In [18]:
version = "v5"

In [19]:
path = os.path.join("/", "Users", "alexandra", "Nextcloud-HTW", "SHARED", "SurfaceAI", "data", "mapillary_images", "training", f"{version}_c0", "metadata", "interrater_reliability")
files = [
    "annotator1.csv",
    "annotator2.csv",
    "annotator3.csv",
]

nostreet_values = ["(mainly) no street visible", "surface / smoothness not visible"]
unsure_revise_value = "unsure - please revise"

In [20]:
df = pd.DataFrame()
for file in files:
    df = pd.concat([df, pd.read_csv(os.path.join(path, file))])

In [21]:
df["image_id"] = df.image.apply(
lambda x: str.split(
    x, "https://freemove.f4.htw-berlin.de/data/local-files/?d=v5/"
)[1]).apply(lambda x: int(str.split(x, ".jpg")[0]))

In [22]:
df.loc[~df.nostreet.isna(), "surface"] = "nostreet"
df.loc[~df.nostreet.isna(), "smoothness"] = "nostreet"

In [23]:
# compare predictions(with nostreet)

# make sure to only use images where all three annotators have annotated
image_id_counts = df.groupby(["image_id"]).size()
image_ids = image_id_counts[image_id_counts == 3].index

In [24]:
grouped_surface = df[df.image_id.isin(image_ids) ].groupby(["image_id", "surface"]).size()

# same surface rating 
round(100* len(grouped_surface[grouped_surface == 3]) / len(grouped_surface), 2)

58.95

In [25]:
# same smoothness rating
grouped_smoothness = df[df.image_id.isin(image_ids) ].groupby(["image_id", "smoothness"]).size()
round(100* len(grouped_smoothness[grouped_smoothness == 3]) / len(grouped_smoothness), 2)
# 30%

26.8

In [26]:
# compare predictions (without nostreet)
image_id_counts = df[df.surface != "nostreet"].groupby(["image_id"]).size()
image_ids = image_id_counts[image_id_counts == 3].index
grouped_surface = df[df.image_id.isin(image_ids) ].groupby(["image_id", "surface"]).size()

# same surface rating (with nostreet)
round(100* len(grouped_surface[grouped_surface == 3]) / len(grouped_surface), 2)

91.94

In [27]:
# same smoothness rating
grouped_smoothness = df[df.image_id.isin(image_ids) ].groupby(["image_id", "smoothness"]).size()
round(100* len(grouped_smoothness[grouped_smoothness == 3]) / len(grouped_smoothness), 2)

31.49

In [28]:
df.annotator.value_counts()

annotator
1    180
5    180
4    180
Name: count, dtype: int64

In [32]:
# Compute Cohen's Kappa
image_id_counts = df[df.surface != "nostreet"].groupby(["image_id"]).size()
image_ids = image_id_counts[image_id_counts == 3].index
annotator1 = 1
annotator2 = 4
annotator3 = 5

for i,j in [(annotator1,annotator2), (annotator1,annotator3), (annotator2,annotator3)]:
            rater1 = df[(df.image_id.isin(image_ids)) & (df.annotator == i)].sort_values(by=["image_id"])
            rater2 = df[(df.image_id.isin(image_ids) )& (df.annotator == j)].sort_values(by=["image_id"])
            kappa = cohen_kappa_score(rater1.surface.tolist(), rater2.surface.tolist())
            print(f"Kappa for surface and annotator {i} and {j}: {round(kappa, 2)}")
            kappa = cohen_kappa_score(rater1.smoothness.tolist(), rater2.smoothness.tolist())
            print(f"Kappa for smoothness and annotator {i} and {j}: {round(kappa, 2)}")
            pearson = np.corrcoef(rater1.smoothness_int, rater2.smoothness_int)[0,1]
            print(f"Pearson for smoothness int and annotator {i} and {j}: {round(pearson, 2)}")
            #kappa = cohen_kappa_score(rater1.smoothness_int.tolist(), rater2.smoothness_int.tolist())
            #print(f"Kappa for smoothness-int and annotator {i} and {j}: {round(kappa, 2)}")

Kappa for surface and annotator 1 and 4: 0.96
Kappa for smoothness and annotator 1 and 4: 0.69
Pearson for smoothness int and annotator 1 and 4: 0.79
Kappa for surface and annotator 1 and 5: 0.95
Kappa for smoothness and annotator 1 and 5: 0.47
Pearson for smoothness int and annotator 1 and 5: 0.75
Kappa for surface and annotator 4 and 5: 0.96
Kappa for smoothness and annotator 4 and 5: 0.42
Pearson for smoothness int and annotator 4 and 5: 0.73


In [ ]:
rater1 = df[(df.image_id.isin(image_ids)) & (df.annotator == annotator1)].sort_values(by=["image_id"])
rater2 = df[(df.image_id.isin(image_ids) )& (df.annotator == annotator2)].sort_values(by=["image_id"])
rater3 = df[(df.image_id.isin(image_ids) )& (df.annotator == annotator3)].sort_values(by=["image_id"])
krippendorfs_alpha_surf = kd.alpha(np.array([
                rater1.surface.tolist(), 
                rater2.surface.tolist(),
                rater3.surface.tolist()]), level_of_measurement='nominal')
krippendorfs_alpha_smooth = kd.alpha(np.array([
                rater1.smoothness.tolist(), 
                rater2.smoothness.tolist(),
                rater3.smoothness.tolist()]), level_of_measurement='nominal')
krippendorfs_alpha_smooth_int = kd.alpha(np.array([
                rater1.smoothness_int.tolist(), 
                rater2.smoothness_int.tolist(),
                rater3.smoothness_int.tolist()]), level_of_measurement='ordinal')

print(f"Krippendorfs alpha for surface: {round(krippendorfs_alpha_surf, 2)}")
print(f"Krippendorfs alpha for smoothness: {round(krippendorfs_alpha_smooth, 2)}")
print(f"Krippendorfs alpha for smoothness as ordinal: {round(krippendorfs_alpha_smooth_int, 2)}")

Krippendorfs alpha for surface: 0.96
Krippendorfs alpha for smoothness: 0.52
Krippendorfs alpha for smoothness as ordinal: 0.74


In [33]:
df.surface.value_counts()

surface
asphalt          202
nostreet         118
paving_stones     71
unpaved           63
concrete          46
sett              40
Name: count, dtype: int64

In [34]:
df.smoothness.value_counts()

smoothness
good            167
nostreet        118
intermediate    104
excellent        85
bad              48
very_bad         18
Name: count, dtype: int64

In [ ]:
grouped_smoothness[grouped_smoothness < 3]

image_id          smoothness  
133711235471285   excellent       2
                  good            1
150388490313514   bad             2
                  very_bad        1
152418293439228   bad             1
                                 ..
1893331670830341  good            2
2978958492337732  bad             1
                  intermediate    2
5471546489584414  excellent       2
                  good            1
Length: 124, dtype: int64

In [31]:
# learning to rank
df["smoothness_int"] = df.smoothness.apply(lambda x: 
                                           0 if x == "ver_bad" 
                                           else 1 if x == "bad" 
                                           else 2 if x == "intermediate" 
                                           else 3 if x == "good" 
                                           else 4)

In [ ]:
df

,annotation_id,annotator,created_at,id,image,lead_time,nostreet,rails,roadtype,smoothness,surface,updated_at,image_id,smoothness_int
0,2191,1,2024-01-25T09:14:06.497773Z,11261,https://freemove.f4.htw-berlin.de/data/local-f...,49.734,NaN,NaN,path (unspecified),very_bad,unpaved,2024-01-25T09:14:06.497819Z,400591394237777,4
1,2192,1,2024-01-25T09:14:21.763658Z,11262,https://freemove.f4.htw-berlin.de/data/local-f...,14.851,NaN,NaN,road(car),excellent,asphalt,2024-01-25T09:14:21.763686Z,343842900417019,4
2,2193,1,2024-01-25T09:14:37.148282Z,11263,https://freemove.f4.htw-berlin.de/data/local-f...,14.906,surface / smoothness not recognizable,NaN,NaN,nostreet,nostreet,2024-01-25T09:14:37.148308Z,311173223724780,4
3,2194,1,2024-01-25T09:15:10.549668Z,11264,https://freemove.f4.htw-berlin.de/data/local-f...,75.187,NaN,NaN,path (unspecified),intermediate,paving_stones,2024-01-25T09:17:49.449167Z,1370581949983309,2
4,2195,1,2024-01-25T09:15:36.713374Z,11265,https://freemove.f4.htw-berlin.de/data/local-f...,29.964,NaN,NaN,road(car),excellent,asphalt,2024-01-25T09:17:04.518672Z,303593837938660,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2726,4,2024-01-25T18:32:41.849150Z,11796,https://freemove.f4.htw-berlin.de/data/local-f...,12.067,NaN,NaN,road(car),good,paving_stones,2024-01-25T18:32:41.849176Z,489332022421464,3
176,2727,4,2024-01-25T18:33:04.995865Z,11797,https://freemove.f4.htw-berlin.de/data/local-f...,22.638,unsure - please revise,NaN,road(car),nostreet,nostreet,2024-01-25T18:33:04.995890Z,911260209606682,4
177,2728,4,2024-01-25T18:33:12.702549Z,11798,https://freemove.f4.htw-berlin.de/data/local-f...,6.991,NaN,NaN,path (unspecified),very_bad,unpaved,2024-01-25T18:33:12.702576Z,176719851431162,4
178,2729,4,2024-01-25T18:33:21.707119Z,11799,https://freemove.f4.htw-berlin.de/data/local-f...,8.473,surface / smoothness not recognizable,NaN,NaN,nostreet,nostreet,2024-01-25T18:33:21.707154Z,1543611642513017,4


In [ ]:
differing_ids = grouped_smoothness[grouped_smoothness < 3].index.get_level_values(0).tolist()
df[df.image_id.isin(differing_ids)].sort_values(by=["image_id", "annotator"])[["annotator", "image_id", "surface", "smoothness"]].to_csv("test_irr.csv")


In [ ]:
import os
import shutil

def copy_files(file_list, destination_folder):
    # Ensure the destination folder exists, create it if needed
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    for file_name in file_list:
        source_path = file_name
        destination_path = os.path.join(destination_folder, os.path.basename(file_name))

        try:
            shutil.copy2(source_path, destination_path)
            #print(f"Copied: {file_name} to {destination_folder}")
        except FileNotFoundError:
            print(f"File not found: {file_name}")
        except Exception as e:
            print(f"Error copying {file_name}: {e}")

# Example usage:
origin_folder = "/Users/alexandra/Nextcloud-HTW/SHARED/SurfaceAI/data/mapillary_images/training/V5_c0/unsorted_images"
file_list = [os.path.join(origin_folder, f"{img_id}.jpg") for img_id in differing_ids]  # Add your file names here
destination_folder = os.path.join(origin_folder, "differing_ratings")

copy_files(file_list, destination_folder)


Copied: /Users/alexandra/Nextcloud-HTW/SHARED/SurfaceAI/data/mapillary_images/training/V5_c0/unsorted_images/133711235471285.jpg to /Users/alexandra/Nextcloud-HTW/SHARED/SurfaceAI/data/mapillary_images/training/V5_c0/unsorted_images/differing_ratings
Copied: /Users/alexandra/Nextcloud-HTW/SHARED/SurfaceAI/data/mapillary_images/training/V5_c0/unsorted_images/133711235471285.jpg to /Users/alexandra/Nextcloud-HTW/SHARED/SurfaceAI/data/mapillary_images/training/V5_c0/unsorted_images/differing_ratings
Copied: /Users/alexandra/Nextcloud-HTW/SHARED/SurfaceAI/data/mapillary_images/training/V5_c0/unsorted_images/150388490313514.jpg to /Users/alexandra/Nextcloud-HTW/SHARED/SurfaceAI/data/mapillary_images/training/V5_c0/unsorted_images/differing_ratings
Copied: /Users/alexandra/Nextcloud-HTW/SHARED/SurfaceAI/data/mapillary_images/training/V5_c0/unsorted_images/150388490313514.jpg to /Users/alexandra/Nextcloud-HTW/SHARED/SurfaceAI/data/mapillary_images/training/V5_c0/unsorted_images/differing_rati